In [1]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [2]:
def format_time(x):
    if not any(i in x for i in ['DNF', 'DNS']):
        if ':' in x:
            return round(float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])), 3) if x != 0 else 0
        else:
            return(round(float(x), 3))
    else:
        return x

In [3]:
def reverse_name(x):
    if str(x) == 'guanyu_zhou':
        name = x.split('_')
        return name[1] + '_' + name[0]
    else:
        return x

In [4]:
races = pd.read_csv(path+'data/races.csv')
qualifying = pd.read_csv(path+'data/qualifying.csv')
constructors = pd.read_csv(path+'data/constructor_standings.csv')
drivers = pd.read_csv(path+'data/driver_standings.csv')
results = pd.read_csv(path+'data/results.csv')
starting_grid = pd.read_csv(path+'data/starting-grid.csv')

In [5]:
results.query('grid == 0').shape

(50, 13)

In [6]:
df1 = pd.merge(races, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time', 'grid'], axis=1)
df2 = pd.merge(df1, drivers, how='left', on=['season', 'round', 'driver']) 
df3 = pd.merge(df2, constructors, how='left', on=['season', 'round', 'constructor'])
df4 = pd.merge(df3, qualifying, how='inner', on=['season', 'round', 'circuit_id', 'driver']).drop(['fastest_lap'], axis=1)
df4.rename(columns = {'fastest_time': 'qual_time', 'grid': 'qualifying_pos'}, inplace=True)
df4.driver = df4.driver.apply(lambda x: reverse_name(x))

final_df = pd.merge(df4, starting_grid, how='inner', on=['season', 'round', 'driver'])
final_df.rename(columns = {'grid': 'starting_grid'}, inplace=True)

final_df.shape

(3651, 23)

In [7]:
final_df.columns

Index(['season', 'round', 'circuit_id', 'country', 'lat', 'long', 'date',
       'driver', 'date_of_birth', 'nationality', 'constructor', 'podium',
       'driver_points', 'driver_wins', 'driver_standings_pos',
       'constructor_points', 'constructor_wins', 'constructor_standings_pos',
       'qualifying_pos', 'qual_time', 'stage', 'q_delta', 'starting_grid'],
      dtype='object')

In [8]:
null_cols = final_df.columns[final_df.isna().any()].tolist()

null_rows = final_df[final_df.isna().any(axis=1)]
null_rows = null_rows[['season', 'round', 'driver', 'podium', 'qual_time', 'stage']]

null_rows.shape

(32, 6)

In [9]:
# fill/drop nulls
### Data is being lossed here

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins', 'constructor_standings_pos']:
            
    final_df[col].fillna(0, inplace=True)
    final_df[col] = final_df[col].map(lambda x: int(x))
    
final_df.dropna(inplace=True)

final_df.shape

(3619, 23)

In [34]:
final_df.to_csv(path+'data/merged.csv', index=False)